In [78]:
#21/6/24 Random Forest Model Creation for Basak Descriptors of Mutagen Dataset

import pandas as pd
import numpy as np
import math
import os
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#Data import
seed = 82

datasets = []
index = []
directory = '/Users/james/Documents/Honours/Data/structdata/Mutagen/chemopy/'
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
        df.drop(columns=['SMILES'], inplace=True)
        df = df.dropna(axis=1)
        datasets.append(df)
        index.append(file)

In [79]:
loops = 0
for df in datasets:
    print(df.isnull().values.any())
    print(index[loops])
    loops = loops + 1

False
mut_kappa.csv
False
mut_basak.csv
False
mut_connectivity.csv
False
mut_constitution.csv
False
mut_moe.csv
False
mut_charge.csv
False
mut_estate.csv
False
mut_molprop.csv
False
mut_topological.csv


In [82]:
#function to calculate various metrics, outputs a list of various metrics with a consistent index
def metriccalc(preds, ytrain):
    correctcount = 0
    fpcount = 0
    tpcount = 0
    tncount = 0
    fncount = 0
    testpos = 0
    testneg = 0
    
    #loop through each item in the predictions, logging positives, negatives and tn/tp/fn/tp
    iterations = 0
    for value in preds:
        testscore = ytrain[iterations]
        if value == 1:
            if testscore != 0:
                testpos = testpos + 1
                correctcount = correctcount + 1
                tpcount = tpcount + 1
            else:
                fpcount = fpcount + 1
                testneg = testneg + 1
        else:
            if testscore != 0:
                testpos = testpos + 1
                fncount = fncount + 1
            else:
                testneg = testneg + 1
                correctcount = correctcount + 1
                tncount = tncount + 1

        iterations = iterations + 1
    
    #calculate a wide swathe of metrics
    netfn = fncount / (fncount + tncount)
    nettn = tncount / (fncount + tncount)
    netacc = correctcount / (fpcount + fncount + tpcount + tncount)
    posacc = tpcount / testpos
    negacc = tncount / testneg
    netfp = fpcount / (fpcount + tpcount)
    nettp = tpcount / (tpcount + fpcount)

    fpr = fpcount / (fpcount + tncount)
    tpr = tpcount / (tpcount + fncount)



    f1 = (2 * tpcount) / ((2 * tpcount) + fpcount + fncount)


    tp = tpcount
    fp = fpcount
    tn = tncount
    fn = fncount

    
    temp = math.sqrt((fp + tn) * (tp + fp) * (tp + fn) * (tn + fn))
    if temp == 0:
        return [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    mcc = ((tp * tn) - (fp * fn)) / temp



    temp = (( ( (tp + fp) * (fp + tn) ) + ( (tp + fn) * (fn + tn) ) ))
    if temp == 0:
        return 0
    kapp =  ( 2 * ((tp * tn) - (fn * fp)) ) / temp
    
    metriclist = [testpos, testneg, fn, tn, tp, fp, netacc, posacc, negacc, fpr, tpr, f1, mcc, kapp]
    
    return metriclist

In [86]:
loops = 0
from sklearn import preprocessing
import numpy as np
for targetdata in datasets:
    trainset, testset = train_test_split(targetdata, test_size=0.5, random_state=seed)
    xtrain = trainset.iloc[:, 1:]
    ytrain = trainset.iloc[:, 0].values
    xtest = testset.iloc[:, 1:]
    ytest = testset.iloc[:, 0].values

#    scaler = preprocessing.StandardScaler().fit(xtrain)
#    xtrain = scaler.transform(xtrain)
    
#    scaler = preprocessing.StandardScaler().fit(xtest)
#    xtest = scaler.transform(xtest)
    
    mcclist = []
    maxval = 0
    for mtry in range(1, 50):
        #using mtry as the adjusted hyperparameter creates a series of random forests
        rf = RandomForestClassifier(n_estimators=mtry, criterion='entropy', max_depth=None, 
                                min_samples_split=2, min_samples_leaf=1, 
                                min_weight_fraction_leaf=0.0, max_features='sqrt', 
                                max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                bootstrap=True, oob_score=False, n_jobs= 4, random_state=seed, 
                                verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)

        model= rf.fit(xtrain, ytrain)
        preds = model.predict(xtest)
        #calculate metric (mcc)
        mcc = metriccalc(preds, ytest)[12]
        mcclist.append(mcc)
        if mcc > maxval:
            maxval = mcc
            bestmetrics = mtry
    #store best model for the given fold and plot the metric vs mcc value
    rf = RandomForestClassifier(n_estimators=bestmetrics, criterion='entropy', max_depth=None, 
                                min_samples_split=2, min_samples_leaf=1, 
                                min_weight_fraction_leaf=0.0, max_features='sqrt', 
                                max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                bootstrap=True, oob_score=False, n_jobs= 4, random_state=seed, 
                                verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)
    print('best mcc of', maxval, 'with an mtry of', bestmetrics)
    #plt.plot(mcclist)

    #plt.xlabel('mtry')
    #plt.ylabel('mcc')
    #plt.legend()
    #plt.show()

    model= rf.fit(xtrain, ytrain)
    preds = model.predict(xtest)
    results = metriccalc(preds, ytest)
    print('for dataset', index[loops], 'validation metrics of:')
    print('positives in data', results[0])
    print('negatives in data', results[1], '\n')
    print('fn count =', results[2])
    print('tn count =', results[3])
    print('tp count =', results[4])
    print('fp count =', results[5], '\n')
    print('net accuracy =', results[6])
    print('positive accuracy =', results[7])
    print('negative accuracy =', results[8], '\n')
    print('fpr =', results[9])
    print('tpr =', results[10], '\n')
    print('f1 score =',results[11])
    print('mcc =',results[12])
    print('cohen Kappa =',results[13], '\n \n')
    loops = loops + 1

best mcc of 0.39195354248497805 with an mtry of 22
for dataset mut_kappa.csv validation metrics of:
positives in data 2093
negatives in data 1650 

fn count = 519
tn count = 1052
tp count = 1574
fp count = 598 

net accuracy = 0.7015762757146674
positive accuracy = 0.7520305781175346
negative accuracy = 0.6375757575757576 

fpr = 0.3624242424242424
tpr = 0.7520305781175346 

f1 score = 0.7381008206330598
mcc = 0.39195354248497805
cohen Kappa = 0.391590509935894 
 

best mcc of 0.4216751150672551 with an mtry of 21
for dataset mut_basak.csv validation metrics of:
positives in data 2093
negatives in data 1650 

fn count = 462
tn count = 1052
tp count = 1631
fp count = 598 

net accuracy = 0.7168047021106064
positive accuracy = 0.7792642140468228
negative accuracy = 0.6375757575757576 

fpr = 0.3624242424242424
tpr = 0.7792642140468228 

f1 score = 0.7547431744562703
mcc = 0.4216751150672551
cohen Kappa = 0.4205080088779158 
 

best mcc of 0.4984993664992035 with an mtry of 29
for dataset

ValueError: Input X contains infinity or a value too large for dtype('float32').

In [61]:
X

NameError: name 'X' is not defined